In [67]:
import pandas as pd
import datacompy
benchmark_file = 'input/benchmark-nomsATC.tsv'
rero_files = ['input/RERO-aligned-2021-11.txt', 'input/RERO-google-extract-20220330.tsv']
rero_concord_file = 'input/20220228_concordances_id_rnv_id_rero_4_headings.csv'

error_file = 'output/erreurs-nomsATC.csv'
missing_file = 'output/manquants-nomsATC.csv'

Import all data into dataframes. Specifying the `dtype` appears necessary because otherwise pandas doesn't know what to do with columns containing both numbers and text.

In [72]:
ouali_data = pd.read_csv(benchmark_file, sep='\t', dtype = str)
rero_data = pd.concat((pd.read_csv(f, sep='\t', encoding = "ISO-8859-1", dtype = str) for f in rero_files))
rero_data.columns =[column.replace(" ", "_") for column in rero_data.columns]
# Remove any duplicate entries between the RERO extracts
rero_data.drop_duplicates(subset=['rero_id'],inplace=True)
rero_concord = pd.read_csv(rero_concord_file, dtype = str)

In [73]:
print("Nombre de concordances validées dans le fichier RERO: ",len(rero_data))

Nombre de concordances validées dans le fichier RERO:  25511


Not sure why the dataframe query doesn't seem to work with strings. Maybe because of extra spaces? Using loc and contains works better. `na=False` is required because that column contains `NaN` data, which we need to convert to false in order to use loc lookup.

In [74]:
display(rero_data.loc[rero_data['author_heading'].str.contains("Dietrich, Andreas", na=False)])
display(rero_concord.loc[rero_concord['main_form'].str.contains("Dietrich, Andreas", na=False)])

,rero_id,idref_id,author_heading,subject_heading
4375,A003180228,257367934,"Dietrich, Andreas",NaN


,id,id_rnv,id_rero,id_rero_a,main_form,repo_id
124357,981023317235702851,(RNV_A)0000071017,(RERO)vtls003180228,NaN,"Dietrich, Andreas",rnv-nz-auth-atc
761184,981023380231902851,(RNV_A)0001220989,(RERO)vtls014190124,NaN,"Dietrich, Andreas, 1976-",rnv-nz-auth-atc


The number in `rero_id` in `rero_data` corresponds to the last part of either `id_rero` or `id_rero_a` in `rero_concord`. All need to be trimmed of their control characters before they can be used as matchpoints.

In [75]:
rero_data['id_rero_join'] = rero_data.rero_id.str.extract('(\d+)')
rero_concord['id_rero_join'] = rero_concord['id_rero'].fillna(rero_concord['id_rero_a'])
rero_concord['id_rero_join'] = rero_concord.id_rero_join.str.extract('(\d+)')
display(rero_data.query('id_rero_join == "003180228"'))
display(rero_concord.query('id_rero_join == "003180228"'))

,rero_id,idref_id,author_heading,subject_heading,id_rero_join
4375,A003180228,257367934,"Dietrich, Andreas",NaN,003180228


,id,id_rnv,id_rero,id_rero_a,main_form,repo_id,id_rero_join
124357,981023317235702851,(RNV_A)0000071017,(RERO)vtls003180228,NaN,"Dietrich, Andreas",rnv-nz-auth-atc,003180228


Now we can try merging the two tables

In [77]:
rero_aligns = pd.merge(rero_data, rero_concord, on='id_rero_join', how="inner")
rero_aligns['source'] = rero_aligns['id']
rero_aligns['cible'] = rero_aligns['idref_id']
display(rero_aligns.query('id_rero_join == "003180228"'))
print('Nombre de concordiances RERO-IdRef validées: ' + str(len(rero_aligns)))

,rero_id,idref_id,author_heading,subject_heading,id_rero_join,id,id_rnv,id_rero,id_rero_a,main_form,repo_id,source,cible
4331,A003180228,257367934,"Dietrich, Andreas",NaN,003180228,981023317235702851,(RNV_A)0000071017,(RERO)vtls003180228,NaN,"Dietrich, Andreas",rnv-nz-auth-atc,981023317235702851,257367934


Nombre de concordiances RERO-IdRef validées: 20372


Now we have a reference file to benchmark against.

Let's look at Ouali's output next:

In [78]:
ouali_data['source'] = ouali_data['id source']
ouali_data['cible'] = ouali_data['id cible']
# .copy() is necessary to avoid SettingWithCopyWarning when using datacompy later
ouali_align = ouali_data.query('`nombre de candidats` != "0" & `décision d\'alignement` == "auto"').copy()
ouali_no_align = ouali_data.query('`nombre de candidats` == "0" & `décision d\'alignement` == "auto"').copy()

print('Alignements: ' + str(len(ouali_align)))
print('Non-alignements: ' + str(len(ouali_no_align)))

Alignements: 641618
Non-alignements: 352076


In [79]:
display(ouali_align)
display(rero_aligns)

,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score max,...,type de cible 2,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3,source,cible
0,rnv-nz-auth-atc,981023336220602851,"Melossi, Dario",NaN,NaN,NaN,NaN,auto,1,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981023336220602851,050194798
2,rnv-nz-auth-atc,981023335918202851,"Meinicke, Michael",NaN,NaN,NaN,NaN,auto,1,0.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981023335918202851,100939848
3,rnv-nz-auth-atc,981023292394002851,"Hrozny, Bedřich",NaN,NaN,NaN,NaN,auto,1,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981023292394002851,079549322
4,rnv-nz-auth-atc,981023401035502851,"Lauzon, Jean",NaN,NaN,NaN,NaN,auto,2,0.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981023401035502851,153084596
5,rnv-nz-auth-atc,981023355887602851,"Dowbiggin, Ian Robert, 1952-",NaN,NaN,NaN,NaN,auto,1,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981023355887602851,031772609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993689,rnv-nz-auth-atc,981023307069702851,"Basedow, Johann-Bernhard",NaN,NaN,NaN,NaN,auto,1,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981023307069702851,06117422X
993690,rnv-nz-auth-atc,981023306039602851,"Ingleby, David",NaN,NaN,NaN,NaN,auto,1,0.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981023306039602851,056761503
993691,rnv-nz-auth-atc,981023349559402851,"Touchard-Lafosse, Georges",NaN,NaN,NaN,NaN,auto,1,0.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981023349559402851,027166333
993692,rnv-nz-auth-atc,981023400255402851,"Desarzens, Victor",NaN,NaN,NaN,NaN,auto,1,0.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981023400255402851,031402569


,rero_id,idref_id,author_heading,subject_heading,id_rero_join,id,id_rnv,id_rero,id_rero_a,main_form,repo_id,source,cible
0,A000000028,171781686,"Altdorf (Deutschland ; 1450-1800, lieu d'éditi...",NaN,000000028,981023300652902851,(RNV_A)0000020879,NaN,A000000028,"Altdorf (Deutschland ; 1450-1800, lieu d'éditi...",rnv-nz-auth-atc,981023300652902851,171781686
1,A000000037,027375188,"Amsterdam (1450-1800, lieu d'édition ou d'impr...",NaN,000000037,981023291273702851,(RNV_A)0000873517,NaN,A000000037,"Amsterdam (1450-1800, lieu d'édition ou d'impr...",rnv-nz-auth-atc,981023291273702851,027375188
2,A000000039,029089565,"Ansbach (1450-1800, lieu d'édition ou d'impres...",NaN,000000039,981023284869802851,(RNV_A)0000956670,NaN,A000000039,"Ansbach (1450-1800, lieu d'édition ou d'impres...",rnv-nz-auth-atc,981023284869802851,029089565
3,A000000042,027522938,"Antwerpen (1450-1800, lieu d'édition ou d'impr...",NaN,000000042,981023284869702851,(RNV_A)0000955971,NaN,A000000042,"Antwerpen (1450-1800, lieu d'édition ou d'impr...",rnv-nz-auth-atc,981023284869702851,027522938
4,A000000047,027220427,"Augsburg (1450-1800, lieu d'édition ou d'impre...",NaN,000000047,981023297672102851,(RNV_A)0000792061,NaN,A000000047,"Augsburg (1450-1800, lieu d'édition ou d'impre...",rnv-nz-auth-atc,981023297672102851,027220427
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20367,A010225830,261385917,"Le Locle (Suisse, NE) - Impasse du Lion-d'Or",NaN,010225830,1010225830,NaN,NaN,A010225830,"Le Locle (Suisse, NE) - Impasse du Lion-d'Or",rnv,1010225830,261385917
20368,A010038347,261393340,"Le Locle (Suisse, NE) - La Bourdonnière",NaN,010038347,1010038348,NaN,NaN,A010038347,"Le Locle (Suisse, NE) - La Bourdonnière",rnv,1010038348,261393340
20369,A010225105,261393561,"Le Locle (Suisse, NE) - Maison de paroisse",NaN,010225105,1010225105,NaN,NaN,A010225105,"Le Locle (Suisse, NE) - Maison de paroisse",rnv,1010225105,261393561
20370,A017772669,261394398,"Le Locle (Suisse, NE) - Montpugin",NaN,017772669,1017772669,NaN,NaN,A017772669,"Le Locle (Suisse, NE) - Montpugin",rnv,1017772669,261394398


In [80]:
common_source = pd.merge(rero_aligns, ouali_align, how="inner", on='source', suffixes=("_rero", "_ouali"), copy=True)
print("Alignements Ouali présents dans fichier RERO: ", len(common_source))

Alignements Ouali présents dans fichier RERO:  4858


In [81]:
common_target = pd.merge(rero_aligns, ouali_align, how="inner", on=['source', 'cible'], suffixes=("_rero", "_ouali"), copy=True)
print("Alignements Ouali validés par RERO: ", len(common_target))
print("Pourcentage validé: ", len(common_target)/len(common_source)*100, "%")

Alignements Ouali validés par RERO:  4699
Pourcentage validé:  96.72704816797035 %


In [82]:
divergences = common_source.loc[~(common_source['cible_rero'] == common_source['cible_ouali'])]
display(divergences[['source','cible_ouali','cible_rero','forme principale cible','main_form']])
divergences.to_csv(error_file,columns=['source','cible_ouali','cible_rero','forme principale cible','main_form'],encoding="UTF-8",index=False)

,source,cible_ouali,cible_rero,forme principale cible,main_form
0,981023292733602851,098879456,117789259,"Abū Zahraẗ, Muḥammad","Abū Zahrah, Muḥammad, 1898-1974"
13,981023285513602851,137125097,030384109,Albert von Augsburg,"Albertus, De Saxonia"
21,981023291275002851,070455317,032474423,Ammonios Saccas 0175?-0242?,"Ammonius, Hermeiou"
22,981023296833702851,152792961,026685493,Anastase le Sinaïte saint 05..-0599,"Anastasius, Sinaita"
29,981023285003102851,030945054,167540831,Antonin saint 1389-1459,"Antonino, archevêque de Florence, saint"
...,...,...,...,...,...
4789,981023369854902851,258939575,258939575,"Boss, Sabine 1966-....","Boss, Sabine"
4816,981023298851002851,253657199,257106049,"Pury, Charles Albert de 1752-1833","Pury, Charles Albert de, 1713-1790"
4829,981023304204802851,076453650,259846554,"Feller, Christian 1943-....","Feller, Christian, expert-comptable"
4831,981023287378202851,150125666,068552963,"Bouyer, A.-C.","Courcier, veuve de Louis, fl. 1811-1821"


In [84]:
missed = pd.merge(rero_aligns, ouali_no_align, how="inner", on='source', suffixes=("_rero", "_ouali"), copy=True)
display(missed[['source','cible_ouali','cible_rero','forme principale cible','main_form']])
missed.to_csv(missing_file,columns=['source','cible_ouali','cible_rero','forme principale cible','main_form'],encoding="UTF-8",index=False)
print("Alignements RERO manqués par Ouali: ", len(missed))

,source,cible_ouali,cible_rero,forme principale cible,main_form
0,981023302389702851,NaN,256648107,NaN,"Corrêa, Ruth Valadares"
1,981023303730702851,NaN,115972463,NaN,"Davydov, I︠U︡riĭ Nikolaevich"
2,981023300411502851,NaN,03394265X,NaN,"Derzhavin, Gavrila Romanovich"
3,981023291573902851,NaN,032211414,NaN,"Font Quer, P"
4,981023289136502851,NaN,03478375X,NaN,"Ganshina, Klavdii︠a︡ Aleksandrovna"
...,...,...,...,...,...
177,981023365104302851,NaN,261383205,NaN,"Eggendorffer, Aloïs"
178,981023357333702851,NaN,261383701,NaN,"Eggendorffer, Aloïs-Wilhelm"
179,981023348417402851,NaN,261219642,NaN,"Staeger, Dieter"
180,981023357657102851,NaN,261383205,NaN,"Eggendorffer, Louis"


Alignements RERO manqués par Ouali:  182


In [85]:
compare = datacompy.Compare(rero_aligns, ouali_align, join_columns=['source', 'cible'], 
                            df1_name='RERO Benchmark', df2_name='Ouali')
print(compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

        DataFrame  Columns    Rows
0  RERO Benchmark       13   20372
1           Ouali       24  641618

Column Summary
--------------

Number of columns in common: 2
Number of columns in RERO Benchmark but not in Ouali: 11
Number of columns in Ouali but not in RERO Benchmark: 22

Row Summary
-----------

Matched on: source, cible
Any duplicates on match values: Yes
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 4,691
Number of rows in RERO Benchmark but not in Ouali: 15,681
Number of rows in Ouali but not in RERO Benchmark: 636,927

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 4,691

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 2
Total number of values which compare unequal: 0

Sample Rows Only in RERO Benchmark (First 10 Columns)
-